### 지원사업공고 update (추가, 수정, 삭제)

In [1]:
import requests
import json

from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

import json
import numpy as np
import faiss

from sentence_transformers import SentenceTransformer
from getpass import getpass
from tqdm import tqdm

from utils.get_text import get_hwp_text, get_pdf_text, get_image_text # pdf, hwp, png에서 텍스트를 추출하는 사용자 지정 함수

In [23]:
# # 기존에 저장되어있는 지원사업 공고 파일 및 지난 공고를 불러옴
# with open('data/bizinfo.json', 'r', encoding='utf-8') as f:
#     data = json.load(f)

# with open('data/bizold.json', 'r', encoding='utf-8') as f:
#     old_infos = json.load(f)

In [24]:
# for _, value in data.items():
#     print(f'{len(value)}')

# for _, value in old_infos.items():
#     print(len(value))

In [25]:
# 사이트 접속 차단을 받지 않기 위한 header 지정
header = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
}

# 기업마당의 해시코드를 저장해놓은 dictionary
hashcode_dict = {'01':'금융',
                '02':'기술',
                '03':'인력',
                '07':'수출',
                '08':'내수',
                '09':'창업',
                '10':'경영',
                '12':'기타'}

# key-value를 반대로 저장
reverse_dict = {value: key for key, value in hashcode_dict.items()}

In [26]:
def download_file(URL, header=header):
    '''지원사업 공고에 있는 파일을 다운로드하는 함수'''
    # bs4로 해당 URL의 html 구조 파싱
    response = requests.get(URL, headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 파일의 정보를 담을 빈 딕셔너리 선언
    file_info = {}

    # 다운로드 파일 링크를 담을 빈 리스트 / 파일명을 담을 빈 문자열 선언
    down_link, down_title = [], ''

    # '본문출력파일'의 파일만 다운로드 하도록 수행하는 반복문 + 조건문
    for idx in range(len(soup.select('div.attached_file_list h3'))):
        if soup.select('div.attached_file_list h3')[idx].text == '본문출력파일':
            down_link = soup.select('div.attached_file_list div.right_btn')[-1].select('a')
            file_path = soup.select_one('div.category span').text.strip()

            # 다운 받을 공고문 파일의 이름을 공고문의 title로 바꿔서 정리
            file_title = soup.select_one('h2.title').text.strip()
            file_type = soup.select('div.attached_file_list div.file_name')[-1].text.strip().split('.')[-1]
            down_title = file_title + '.' + file_type
        else:
            continue

    # 다운로드 파일링크는 내부 url이기 때문에 앞에 도메인 주소를 붙여줘야 함.
    base_url = 'https://www.bizinfo.go.kr'
    down_url = base_url + down_link[1].attrs['href']
    

    # 다운로드된 파일을 저장할 파일 경로
    save_path = f'C:/Users/every/Desktop/Project/team_project/SeSAC_Final/AI-for-Creating-Business-Plan-Drafts/{file_path}/{down_title}'

    # HTTP 요청을 보내 파일 다운로드
    response = requests.get(down_url, stream=True)
    try:
        with open(save_path, "wb") as file:
            for chunk in response.iter_content(1024):  # 1024바이트씩 저장
                file.write(chunk)
    except:
        print(f"파일 다운로드 실패. 상태 코드: {response.status_code}")
        print(f"다운로드 실패한 공고 명 : {down_title}")


    # 파일의 정보를 딕셔너리 형태로 담음
    # 프레임으로 관리 및 공고 수정 사항을 빠르게 탐색하기 위한 정보 모음집
    file_info['지원사업 공고명'] = file_title
    file_info['소관부처·지자체'] = soup.select('div.view_cont li div.txt')[0].text.strip()
    file_info['사업수행기관'] = soup.select('div.view_cont li div.txt')[1].text.strip()
    file_info['신청기간'] = soup.select('div.view_cont li div.txt')[2].text.strip().replace("\r", " ").replace("\n", "").replace("\t", "")
    file_info['공고파일명'] = file_path + '/' + down_title

    return file_info, file_path

def fetch_links(hashcode, header=header):
    '''기업마당의 메인에 있는 공고 각각의 URL을 리스트로 반환'''

    # 기업마당의 공고 URL은 BASE_URL + td class='txt_l a'의 형식으로 되어 있음
    BASE_URL = 'https://www.bizinfo.go.kr/web/lay1/bbs/S1T122C128/AS/74/'
    idx = 1         # 페이지를 넘기기 위한 인덱스
    links = []      # 링크를 정리해둘 리스트 선언

    # 공고가 없는 페이지가 나올 때까지 반복문
    while True:
        URL = f'https://www.bizinfo.go.kr/web/lay1/bbs/S1T122C128/AS/74/list.do?hashCode={hashcode}&cpage={idx}'
        response = requests.get(URL, headers=header)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 공고가 없는 페이지가 나올 시 반복문 탈출
        if len(soup.select('div.sub_cont tr')) == 2:
            break
        
        # links에 URL 저장
        # 해당 페이지에 있는 전체 URL에 대하여 실행
        links += [BASE_URL + link.attrs['href'] for link in soup.select('td.txt_l a')]

        # 페이지 넘김
        idx += 1

    return links

def update_fetch_links(hashcode, header=header):
    '''기업마당의 메인에 있는 공고 각각의 URL을 리스트로 반환'''

    # 기업마당의 공고 URL은 BASE_URL + td class='txt_l a'의 형식으로 되어 있음
    BASE_URL = 'https://www.bizinfo.go.kr/web/lay1/bbs/S1T122C128/AS/74/'
    idx = 1         # 페이지를 넘기기 위한 인덱스
    links, titles = [], []     # 링크를 정리해둘 리스트 선언

    # 공고가 없는 페이지가 나올 때까지 반복문
    while True:
        URL = f'https://www.bizinfo.go.kr/web/lay1/bbs/S1T122C128/AS/74/list.do?hashCode={hashcode}&cpage={idx}'
        response = requests.get(URL, headers=header)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 공고가 없는 페이지가 나올 시 반복문 탈출
        if len(soup.select('div.sub_cont tr')) == 2:
            break
        
        # links에 URL 저장
        # 해당 페이지에 있는 전체 URL에 대하여 실행
        links += [BASE_URL + link.attrs['href'] for link in soup.select('td.txt_l a')]
        titles += [title.text.strip() for title in soup.select('td.txt_l a')]

        # 페이지 넘김
        idx += 1

    return links, titles

In [27]:
# # 오늘 날짜 가져오기
# today = datetime.today().date()        # 현재 날짜 (YYYY-MM-DD) # 실험상 임의로 오늘의 날짜를 1일 뒤로 지정

# # 마감기한이 끝났거나 혹은 기한이 저장되어 있지 않은 공고의 삭제
# for key, value in tqdm(list(data.items())):  # 딕셔너리 항목을 리스트로 변환하여 순회 (수정 가능하게)

#     links, titles = update_fetch_links(reverse_dict[key])  # 기존의 DB와 비교할 오늘의 공고 목록
#     now_titles = [] # 현재 DB에 있는 항목을 저장할 리스트

#     expired_items = []  # 삭제할 항목을 저장할 리스트
    
#     for idx in value[:]:  # 원본 리스트를 복사하여 순회

#         now_titles.append(idx['지원사업 공고명']) # 현재 공고의 타이틀을 저장

#         if idx['신청기간'][0] == '2':  # 마감 기한이 숫자로 시작하는 경우
#             deadline = datetime.strptime(idx['신청기간'][-10:], "%Y.%m.%d").date()
#             if deadline < today:  # 마감 기한이 현재 날짜보다 이전이라면
#                 old_infos[key].append(idx)  # 기존 정보 저장
#                 expired_items.append(idx)  # 삭제할 항목 저장
    

#         else:  # 마감 기한이 날짜가 아닌 경우 (예산 소진시까지, 추후 공지, 상시 접수 등)
#             if idx['지원사업 공고명'] not in titles:
#                 old_infos[key].append(idx)  # 기존 정보 저장
#                 expired_items.append(idx)  # 삭제할 항목 저장

        
#     # 만료된 항목을 기존 리스트에서 제거
#     for item in expired_items:
#         value.remove(item)
    
#     # 만약 해당 key의 리스트가 비었다면, 아예 삭제
#     if not value:
#         del data[key]

#     # 새로운 항목에 대하여 기존 딕셔너리에 저장
#     for i in range(len(titles)):
#         if titles[i] not in now_titles:
#             print(titles[i])
#             file_info, file_path = download_file(links[i])
#             data[file_path].append(file_info)

In [28]:
# for _, value in data.items():
#     print(len(value))

# for _, value in old_infos.items():
#     print(len(value))

In [29]:
# with open('data/bizinfo.json', 'w', encoding='utf-8') as f:
#     json.dump(data, f, indent=4, ensure_ascii=False)

# with open('data/bizold.json', 'w', encoding='utf-8') as f:
#     json.dump(old_infos, f, indent=4, ensure_ascii=False)

### 금융에 대해서만 update

In [30]:
# 기존에 저장되어있는 지원사업 공고 파일 및 지난 공고를 불러옴
with open('data/bizinfo_2.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

with open('data/bizold_2.json', 'r', encoding='utf-8') as f:
    old_infos = json.load(f)

In [31]:
# 개수 확인용 출력
print(f'전날 금융의 공고 개수 : {len(data['금융'])}, 지난 공고의 개수 : {len(old_infos['금융'])}')

전날 금융의 공고 개수 : 166, 지난 공고의 개수 : 0


In [32]:
# 오늘 날짜 가져오기
today = datetime.today().date()        # 현재 날짜 (YYYY-MM-DD) # 실험상 임의로 오늘의 날짜를 1일 뒤로 지정

# 마감기한이 끝났거나 혹은 기한이 저장되어 있지 않은 공고의 삭제
for key, value in tqdm(list(data.items())):  # 딕셔너리 항목을 리스트로 변환하여 순회 (수정 가능하게)

    links, titles = update_fetch_links(reverse_dict[key])  # 기존의 DB와 비교할 오늘의 공고 목록
    now_titles = [] # 현재 DB에 있는 항목을 저장할 리스트

    expired_items = []  # 삭제할 항목을 저장할 리스트
    
    for idx in value[:]:  # 원본 리스트를 복사하여 순회

        now_titles.append(idx['지원사업 공고명']) # 현재 공고의 타이틀을 저장

        if idx['신청기간'][0] == '2':  # 마감 기한이 숫자로 시작하는 경우
            deadline = datetime.strptime(idx['신청기간'][-10:], "%Y.%m.%d").date()
            if deadline < today:  # 마감 기한이 현재 날짜보다 이전이라면
                old_infos[key].append(idx)  # 기존 정보 저장
                expired_items.append(idx)  # 삭제할 항목 저장
    

        else:  # 마감 기한이 날짜가 아닌 경우 (예산 소진시까지, 추후 공지, 상시 접수 등)
            if idx['지원사업 공고명'] not in titles:
                old_infos[key].append(idx)  # 기존 정보 저장
                expired_items.append(idx)  # 삭제할 항목 저장

        
    # 만료된 항목을 기존 리스트에서 제거
    for item in expired_items:
        value.remove(item)
    
    # 만약 해당 key의 리스트가 비었다면, 아예 삭제
    if not value:
        del data[key]

    # 새로운 항목에 대하여 기존 딕셔너리에 저장
    for i in range(len(titles)):
        if titles[i] not in now_titles:
            print(titles[i])
            file_info, file_path = download_file(links[i])
            data[file_path].append(file_info)
    
    break

  0%|          | 0/8 [00:11<?, ?it/s]


In [33]:
# 개수 확인용 출력
print(f'전날 금융의 공고 개수 : {len(data['금융'])}, 지난 공고의 개수 : {len(old_infos['금융'])}')

전날 금융의 공고 개수 : 166, 지난 공고의 개수 : 0


In [34]:
with open('data/bizinfo_2.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

with open('data/bizold_2.json', 'w', encoding='utf-8') as f:
    json.dump(old_infos, f, indent=4, ensure_ascii=False)